# Глубокое обучение и обработка естественного языка

## Домашняя работа №5

Загрузить набор данных Lenta.ru с помощью пакета Corus

1. Обучить LDA модель, постараться подобрать адекватные параметры (num_topics, passes, alpha, iterations…) - 4 балла
2. Визуализировать результаты работы LDA с помощью pyLDAvis - 2 балла
3. Посчитать внутренние метрики обученных моделей LDA (с разными параметрами) и сравнить, соответствует ли метрика визуальному качеству работы моделей - 2 балла
4. Обучить модель BigARTM, использовать не менее двух регуляризаторов, оценить качество с помощью метрик - 5 баллов
5. Реализовать визуализацию топиков BigARTM через pyLDAvis - 4 балла
6. Обеспечена воспроизводимость решения: зафиксированы random_state, ноутбук воспроизводится от начала до конца без ошибок - 2 балла
7. Соблюден code style на уровне pep8 и On writing clean Jupyter notebooks - 1 балл

**Примечание:** подбирать параметры теметической модели можно также, как и для любой другой модели - на кроссвалидации, ориентируясь на метрики качества

In [ ]:
!pip install bigartm corus pymorphy2 razdel pyLDAvis

In [ ]:
import re
from pprint import pprint
from collections import Counter
import itertools
from tqdm.notebook import tqdm
import warnings, logging
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.colors as mcolors
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords

import gensim
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel

import artm

import razdel
from corus import load_lenta

import pymorphy2
from bs4 import BeautifulSoup

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

nltk.download('punkt')
nltk.download('stopwords')

warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
SEED = 2023

np.random.seed(SEED)

In [3]:
# функция, очистка от html разметки
def clean_html_bs4(text_data):
  soup = BeautifulSoup(text_data, 'lxml')
  return soup.get_text()

# функция, очистка от мусора, нормализация и лемматизация
def tokenize(text, stopwords, need_lemmatize=False):
  result = []
  sentences = [item.text for item in razdel.sentenize(str(text))]

  for sentence in sentences:
    text = sentence.lower()
    text = clean_html_bs4(text)
    text = re.sub(r"\s+", ' ', text)

    tokens = [item.text for item in  razdel.tokenize(text)]
    tokens = [re.sub("[^а-яА-Яa-zA-Z]", ' ', item) for item in tokens]

    if need_lemmatize:
      tokens = [analyzer.parse(token)[0].normal_form for token in tokens if token not in stopwords  and ' ' not in token and len(token) > 2]
      tokens = [token for token in tokens if token not in lemmatized_sw]
    tokens = [re.sub(r"ё", "е", token) for token in tokens]
    result.extend(tokens)
  return result

# функция получение сочетаний слов
def get_ngrams(texts_out):
    texts_out = [trigram[bigram[doc]] for doc in texts_out]
    return texts_out

## Загрузка данных

In [ ]:
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

In [42]:
path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)
dataset = [next(records).text for i in range(1000)]

## Предобработка

In [43]:
sw = stopwords.words('russian')
additional_sw =  'мои оно мной мною мог могут мор мое мочь оба нам нами ними однако нему никуда наш нею неё наша наше наши очень отсюда вон вами ваш ваша ваше ваши весь всем всеми вся ими ею будем будете будешь буду будь будут кому кого которой которого которая которые который которых кем каждое каждая каждые каждый кажется та те тому собой тобой собою тобою тою хотеть хочешь свое свои твой своей своего своих твоя твоё сама сами теми само самом самому самой самого самим самими самих саму чему тебе такое такие также такая сих тех ту эта это этому туда этим этими этих абы аж ан благо буде вроде дабы едва ежели затем зато ибо итак кабы коли коль либо лишь нежели пока покамест покуда поскольку притом причем пускай пусть ровно сиречь словно также точно хотя чисто якобы '
pronouns = 'я мы ты вы он она оно они себя мой твой ваш наш свой его ее их то это тот этот такой таков столько весь всякий сам самый каждый любой иной другой кто что какой каков чей сколько никто ничто некого нечего никакой ничей нисколько кто-то кое-кто кто-нибудь кто-либо что-то кое-что что-нибудь что-либо какой-то какой-либо какой-нибудь некто нечто некоторый некий'
conjunctions = 'что чтобы как когда ибо пока будто словно если потому что оттого что так как так что лишь только как будто с тех пор как в связи с тем что для того чтобы кто как когда который какой где куда откуда'
digits = 'ноль один два три четыре пять шесть семь восемь девять десять одиннадцать двенадцать тринадцать четырнадцать пятнадцать шестнадцать семнадцать восемнадцать девятнадцать двадцать тридцать сорок пятьдесят шестьдесят семьдесят восемьдесят девяносто сто'
modal_words = 'вероятно возможно видимо по-видимому кажется наверное безусловно верно  действительно конечно несомненно разумеется'
particles = 'да так точно ну да не ни неужели ли разве а что ли что за то-то как ну и ведь даже еще ведь уже все все-таки просто прямо вон это вот как словно будто точно как будто вроде как бы именно как раз подлинно ровно лишь только хоть всего исключительно вряд ли едва ли'
prepositions = 'близ  вблизи  вдоль  вокруг  впереди  внутрь  внутри  возле  около  поверх  сверху  сверх  позади  сзади  сквозь  среди  прежде  мимо  вслед  согласно  подобно  навстречу  против  напротив  вопреки  после  кроме  вместе  вдали  наряду  совместно  согласно  нежели вроде от бишь до без аж тех раньше совсем только итак например из прямо ли следствие а поскольку благо пускай благодаря случае затем притом также связи время при чтоб просто того невзирая даром вместо точно покуда тогда зато ради ан буде прежде насчет раз причине тому так даже исходя коль кабы более ровно либо помимо как-то будто если словно лишь бы и не будь пор тоже разве чуть как хотя наряду потому пусть в равно между сверх ибо на судя то чтобы относительно или счет за но сравнению причем оттого есть когда уж ввиду тем для дабы чем хоть с вплоть скоро едва после той да вопреки ежели кроме сиречь же коли под абы несмотря все пока покамест паче прямо-таки перед что по вдруг якобы подобно'
evaluative = 'наиболее наименее лучший больший высший низший худший более менее'

sw.extend(additional_sw.split())
sw.extend(pronouns.split())
sw.extend(conjunctions.split())
sw.extend(digits.split())
sw.extend(modal_words.split())
sw.extend(particles.split())
sw.extend(prepositions.split())
sw.extend(evaluative.split())
sw = list(set(sw))

Очистка от мусора, нормализация и лемматизация

In [44]:
need_preprocess = True

if need_preprocess:
  analyzer = pymorphy2.MorphAnalyzer()
  lemmatized_sw = [analyzer.parse(word)[0].normal_form for word in sw]
  tokenized = [tokenize(text, stopwords=sw, need_lemmatize=True) for text in dataset]

<ipython-input-3-575bae51654c>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text_data, 'lxml')


Обнаружение сочетаний слов

In [8]:
bigram = gensim.models.Phrases(tokenized, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[tokenized], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [9]:
tokenized = get_ngrams(tokenized)

## Создание словаря

In [10]:
id2word = corpora.Dictionary(tokenized)
corpus = [id2word.doc2bow(text) for text in tokenized]

## Модели

### LDA

In [11]:
models = []
params =  {'num_topics': [3, 4, 5, 6, 7, 8], 'chunksize': [1, 64], 'alpha': ['symmetric', 'auto'], 'iterations': [50, 150]}

In [12]:
params = list(params.values())
params_tuples = []

for element in itertools.product(*params):
    params_tuples.append(element)

In [13]:
%%time

for params in tqdm(params_tuples):
  model = gensim.models.ldamodel.LdaModel(
      corpus=corpus,
      id2word=id2word,
      num_topics=params[0],
      random_state=SEED,
      chunksize=params[1],
      alpha=params[2],
      iterations=params[3],
      per_word_topics=True,
  )
  models.append(model)

  0%|          | 0/48 [00:00<?, ?it/s]

CPU times: user 5min 36s, sys: 849 ms, total: 5min 37s
Wall time: 5min 47s


#### Метрики

In [14]:
perplexity = np.zeros(len(models))
coherence_cv = np.zeros(len(models))
coherence_umass = np.zeros(len(models))

In [15]:
for i in range(len(models)):
  cm_umass = CoherenceModel(model=models[i], corpus=corpus, coherence='u_mass')
  cm_cv = CoherenceModel(model=models[i], texts=tokenized, coherence='c_v')

  perplexity[i] = model.log_perplexity(corpus)
  coherence_umass[i] = cm_umass.get_coherence()
  coherence_cv[i] = cm_cv.get_coherence()

In [16]:
print("Перплексия моделей: ", perplexity)
print("Модель с наименьшей перплексией: ", perplexity.argmin())

Перплексия моделей:  [-9.62004984 -9.62004977 -9.62004974 -9.62004965 -9.62004968 -9.62004976
 -9.6200499  -9.62004982 -9.6200501  -9.62004974 -9.62004977 -9.62004984
 -9.62004975 -9.62004981 -9.62004977 -9.62004984 -9.6200499  -9.62004968
 -9.62004975 -9.62004984 -9.62004972 -9.62004974 -9.62004979 -9.62004972
 -9.6200498  -9.62004968 -9.6200497  -9.6200498  -9.62004972 -9.6200496
 -9.62004972 -9.62004985 -9.6200497  -9.62004977 -9.62004971 -9.62004984
 -9.62004986 -9.62004991 -9.62004963 -9.62005016 -9.62004981 -9.62004975
 -9.62004971 -9.62004969 -9.62005006 -9.62004963 -9.62004965 -9.62004984]
Модель с наименьшей перплексией:  39


Перплексия незначительно различается среди моделей.

In [18]:
print("Когеренция_umass моделей: ", coherence_umass)
print("Модель с наибольшей когеренцией_umass: ", coherence_umass.argmax())

Когеренция_umass моделей:  [-4.1029411  -3.40750377 -3.2368754  -3.4825014  -1.80834553 -2.03264776
 -1.77730325 -2.02798511 -3.33515276 -4.04788001 -3.03343124 -3.5837878
 -2.05049295 -2.04730375 -2.0420132  -2.04881579 -3.35389492 -3.0542244
 -3.2995249  -3.28822717 -2.47226503 -2.6209805  -2.55822179 -2.5531511
 -3.98637587 -3.66472304 -3.72674939 -3.36186509 -3.5269557  -3.21623498
 -3.49496879 -3.16464948 -4.04133101 -5.00643765 -4.21976563 -4.52565758
 -3.52906184 -3.11477342 -3.73488001 -3.63279827 -4.73911072 -4.86757693
 -5.01365659 -4.50719938 -4.4971581  -4.66891215 -4.44763559 -4.88478773]
Модель с наибольшей когеренцией_umass:  6


Когеренция_umass наибольшая для модели 6

In [19]:
print("Когеренция_cv моделей: ", coherence_cv)
print("Модель с наибольшей когеренцией_cv: ", coherence_cv.argmax())

Когеренция_cv моделей:  [0.34034298 0.3327381  0.34384493 0.31384925 0.33642255 0.31086391
 0.33091302 0.31033347 0.29547499 0.28838857 0.27196326 0.28813916
 0.34977802 0.29798003 0.34526834 0.28911739 0.28427611 0.28304526
 0.27462981 0.30536449 0.28696541 0.2574433  0.27528954 0.25784931
 0.24995087 0.27668541 0.25858893 0.26662589 0.30700818 0.30624479
 0.29927827 0.30420272 0.26288427 0.28568002 0.26412417 0.27738913
 0.34100967 0.34330981 0.35102982 0.32440234 0.23445493 0.27465422
 0.29857517 0.26658075 0.33643428 0.33111079 0.32819079 0.33223833]
Модель с наибольшей когеренцией_cv:  38


Когеренция_cv незначительно различается среди моделей

In [20]:
params_tuples[6]

(3, 64, 'auto', 50)

In [21]:
model = models[6]
pprint(model.print_topics())

[(0,
  '0.013*"год" + 0.008*"россия" + 0.008*"российский" + 0.006*"декабрь" + '
  '0.004*"человек" + 0.004*"сообщать" + 0.004*"слово" + 0.004*"стать" + '
  '0.004*"страна" + 0.004*"заявить"'),
 (1,
  '0.013*"год" + 0.010*"россия" + 0.005*"компания" + 0.005*"российский" + '
  '0.005*"страна" + 0.005*"украина" + 0.005*"слово" + 0.005*"заявить" + '
  '0.004*"процент" + 0.004*"украинский"'),
 (2,
  '0.007*"год" + 0.003*"женщина" + 0.003*"сообщать" + 0.003*"ребенок" + '
  '0.003*"стать" + 0.003*"мужчина" + 0.003*"фотография" + 0.003*"человек" + '
  '0.003*"дом" + 0.003*"пользователь"')]


#### Визуализация

In [24]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(model, corpus, dictionary=model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.030941 -0.103386       1        1  37.268623
1     -0.109027  0.070964       2        1  35.613893
2      0.139968  0.032422       3        1  27.117485, topic_info=              Term        Freq       Total Category  logprob  loglift
21          россия  721.000000  721.000000  Default  30.0000  30.0000
421        украина  215.000000  215.000000  Default  29.0000  29.0000
1361    украинский  173.000000  173.000000  Default  28.0000  28.0000
915       компания  256.000000  256.000000  Default  27.0000  27.0000
15         процент  198.000000  198.000000  Default  26.0000  26.0000
...            ...         ...         ...      ...      ...      ...
283          новый   60.153507  227.623203   Topic3  -6.1882  -0.0258
172   американский   56.127926  163.906794   Topic3  -6.2574   0.2333
282      несколько   52.217483  139.032741   Topic3  -6.3296   0.3257
37          тысяча   53.862114  196.831033   Topic3  -6.2986   0.0091
26           слово   55.170238  402.346361   Topic3  -6.2746  -0.6819

[258 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
4950      1  0.038647        агрессия
4950      2  0.966174        агрессия
4950      3  0.038647        агрессия
172       1  0.512486    американский
172       2  0.146425    американский
...     ...       ...             ...
2892      2  0.942642       экономика
4003      1  0.059452           экран
4003      2  0.029726           экран
4003      3  0.921511           экран
1630      1  0.979827  этап_кубок_мир

[505 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])

<Figure size 1000x800 with 0 Axes>

### BigARTM

#### Векторизация текста

In [45]:
tokenized = [' '.join(text) for text in tokenized]

In [46]:
cv = CountVectorizer(max_features=5000, max_df=0.8, min_df=2, ngram_range=(1, 2))
n_wd = np.array(cv.fit_transform(tokenized).todense()).T

In [53]:
token_list = list(cv.vocabulary_.keys())
bv = artm.BatchVectorizer(data_format='bow_n_wd', n_wd=n_wd, vocabulary=token_list)

#### Модель

In [79]:
def objective(space):
  model = artm.ARTM(num_topics=space['num_topics'], dictionary=bv.dictionary, cache_theta=True)

  model.scores.add(artm.PerplexityScore(name='perplexity_score', dictionary=bv.dictionary))
  model.scores.add(artm.SparsityPhiScore(name='sparsity_phi_score'))
  model.scores.add(artm.SparsityThetaScore(name='sparsity_theta_score'))
  model.scores.add(artm.TopTokensScore(name='top_tokens_score', num_tokens=10))

  model.regularizers.add(
      artm.SmoothSparsePhiRegularizer(
        name='SparsePhi',
        tau=space['phi_tau']
    ),
  ) # Сглаживание распределений терминов в темах. Используется для выделения фоновых тем, собирающих общую лексику языка или общую лексику данной коллекции.
  model.regularizers.add(
      artm.SmoothSparseThetaRegularizer(
          name='SparseTheta',
          tau=space['theta_tau']
      ),
  ) # Сглаживание распределений тем в документах. Используется для выделения фоновых слов в каждом документах.
  model.regularizers.add(
      artm.DecorrelatorPhiRegularizer(
        name='DecorrelatorPhi',
        tau=space['decorrelation_tau']
      ),
  ) # Декоррелирование распределений терминов в темах. Используется для повышения различности лексических ядер предметных тем.

  model.fit_offline(bv, num_collection_passes=30)

  perplexity_score = model.score_tracker["perplexity_score"].last_value
  print("Perplexity:", perplexity_score)
  return {'loss': perplexity_score, 'status': STATUS_OK }

In [80]:
topics_to_check =  [10, 15, 20]

In [81]:
def run_hyperparams_search():

  space={
       'num_topics': hp.choice('num_topics', topics_to_check),
       'phi_tau': hp.uniform('SparsePhi', -1, 1),
       'theta_tau': hp.uniform('SparseTheta', -1, 1),
       'decorrelation_tau': hp.uniform('DecorrelatorPhi', 1e+2, 1e+5),
  }

  trials = Trials()

  best_hyperparams = fmin(
      fn=objective,
      space=space,
      algo=tpe.suggest,
      max_evals=20,
      trials=trials,
  )
  return best_hyperparams

In [82]:
%%time
best = run_hyperparams_search()

Perplexity:
1275.1829833984375
Perplexity:
1371.995361328125
Perplexity:
1113.2501220703125
Perplexity:
1512.918212890625
Perplexity:
1289.5155029296875
Perplexity:
1446.0721435546875
Perplexity:
1395.7901611328125
Perplexity:
1445.3197021484375
Perplexity:
1419.489501953125
Perplexity:
950.31689453125
Perplexity:
1332.6915283203125
Perplexity:
1293.1397705078125
Perplexity:
1358.0950927734375
Perplexity:
1063.567626953125
Perplexity:
1618.0601806640625
Perplexity:
1317.1717529296875
Perplexity:
1231.0284423828125
Perplexity:
1134.1470947265625
Perplexity:
1392.762451171875
Perplexity:
1522.552978515625
100%|██████████| 20/20 [01:44<00:00,  5.22s/trial, best loss: 950.31689453125]
CPU times: user 1min 39s, sys: 1.83 s, total: 1min 41s
Wall time: 1min 44s


In [83]:
best

{'DecorrelatorPhi': 9493.307264858711,
 'SparsePhi': 0.19471652297832676,
 'SparseTheta': -0.7233471767323558,
 'num_topics': 2}

In [85]:
best = {'DecorrelatorPhi': 9493.307264858711,
 'SparsePhi': 0.19471652297832676,
 'SparseTheta': -0.7233471767323558,
 'num_topics': 2}

In [86]:
def fit_model(seed=SEED):
  model = artm.ARTM(num_topics=topics_to_check[best['num_topics']], dictionary=bv.dictionary, cache_theta=True, seed=seed)
  model.scores.add(artm.PerplexityScore(name='perplexity_score',
                                        dictionary=bv.dictionary))

  model.scores.add(artm.SparsityPhiScore(name='sparsity_phi_score'))
  model.scores.add(artm.SparsityThetaScore(name='sparsity_theta_score'))
  model.scores.add(artm.TopTokensScore(name='top_tokens_score', num_tokens=10))

  model.regularizers.add(artm.SmoothSparsePhiRegularizer(name='SparsePhi', tau=best['SparsePhi'])) # сглаживание/разреживание матрицы Phi
  model.regularizers.add(artm.SmoothSparseThetaRegularizer(name='SparseTheta', tau=best['SparseTheta'])) # сглаживание/разреживание матрицы Theta
  model.regularizers.add(artm.DecorrelatorPhiRegularizer(name='DecorrelatorPhi', tau=best['DecorrelatorPhi'])) # сделать темы более разнообразными

  model.fit_offline(bv, num_collection_passes=30)
  return model

In [ ]:
%%time

model = fit_model()

#### Метрики

In [88]:
model.score_tracker["perplexity_score"].last_value

973.3284912109375

In [89]:
model.score_tracker["sparsity_phi_score"].last_value

0.0

In [90]:
model.score_tracker["sparsity_theta_score"].last_value

0.9064000248908997

In [91]:
num_words = 10

In [92]:
word_topics = []
for topic_name in model.topic_names:
  temp = []
  for word in model.score_tracker["top_tokens_score"].last_tokens[topic_name]:
    temp.append(word)
  word_topics.append(temp)

In [93]:
def td_score(topics):
  if topics is None:
    return 0
  if num_words > len(topics[0]):
    raise Exception('Words in topics are less than ' + str(num_words))
  else:
    unique_words = set()
    for topic in topics:
      unique_words = unique_words.union(set(topic[:num_words]))
    td = len(unique_words) / (num_words * len(topics))
    return td

In [94]:
td_score(word_topics)

0.65

In [95]:
docs = []
for row in n_wd.T:
  docs.append([token_list[i] for i, value in enumerate(row) if value > 0])

In [96]:
dictionary = corpora.Dictionary(docs)

In [97]:
coherence_model = CoherenceModel(topics=word_topics, texts=docs, dictionary=dictionary, coherence='c_v')
coherence = coherence_model.get_coherence()
coherence

0.522821910880704

#### Визуализация

In [98]:
def prepare_vis_data():
    phi = model.get_phi()
    theta = model.get_theta().to_numpy().T
    theta = theta / theta.sum(axis=1, keepdims=1)
    data = {'topic_term_dists': phi.to_numpy().T,
            'doc_topic_dists': theta,
            'doc_lengths': n_wd.sum(axis=0).tolist(),
            'vocab': phi.T.columns,
            'term_frequency': n_wd.sum(axis=1).tolist()}
    return data

In [99]:
model_data = prepare_vis_data()
model_vis = pyLDAvis.prepare(**model_data)

In [100]:
pyLDAvis.save_html(model_vis, 'lenta_bigartm_vis.html')
pyLDAvis.display(model_vis)

In [101]:
!pip freeze > requirements.txt